In [ ]:
!pip install comet_ml

# 1. Import thư viện

In [25]:
import random

random.seed(42)
print(random.randint(1, 10))  # Lần nào cũng ra 2
print(random.randint(1, 10))  # Lần nào cũng ra 1


2
1


In [ ]:
from comet_ml import Experiment, ExistingExperiment

# Khởi tạo Comet
quality = 40
patch = 1
imgsz = 128
batch_size = 8
isr_output = True

if imgsz == 128 and isr_output == True:
    previous_key = '4f97aded96b7428b9c462cbd636b17a6'
    weight_path = f'/kaggle/input/tamsadsadas/best_iqe_40_{imgsz}_isr_output.pth'
elif imgsz == 64 and isr_output == True:
    previous_key = '3bdcae67caa24c2d89dca167aef9cbde'
    weight_path = f'/kaggle/input/tamsadsadas/best_iqe_40_{imgsz}_isr_output.pth'
experiment = ExistingExperiment(
    previous_experiment = previous_key,
    api_key="mkfmxVIjacb8h74qKO6NzPdPN",
    project_name="cisr-project",
    workspace="dangdinh17"          # Tên workspace trên Comet
)


experiment.set_name(f"Final IQE + ISR Training Run at size of {imgsz}")
# 

In [ ]:
import cv2
import numpy as np
import os
from PIL import Image
import shutil
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim as optim
from torchvision import transforms
import torchsummary
from torch.utils.data import DataLoader, Dataset
from torch.nn.parallel import DataParallel
from torch.cuda.amp import autocast, GradScaler
import torch.nn.functional as F
from torchvision.utils import save_image

from tqdm import tqdm
import sys
sys.path.append('/kaggle/input/full/pytorch/default/3')
from e2dsr import *
from vdsr import *

from hqsr import *
from vdsr import *
from srresnet import *
from sr_model import *
from vdsr import *
from utils import *
from srcnn import *
from edsr import *
from iqe import *
from network_swinir import *
import time
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import random

# Tạo data

In [ ]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
device = torch.device('cuda' if torch.cuda.is_available() else 'cuda')
max_training_time = 11 * 3600 +  50 * 60  # 11 giờ 50 phút tính bằng giây
start_training_time = time.time()  # Ghi lại thời gian bắt đầu


In [ ]:
# img_dir = '/kaggle/input/div2k/DIV2K/HR'
img_dir = '/kaggle/input/test600/test1_600x600/images'

# for quality in [20]:
types = 'test'
# output = f'compress/quality_compress_{quality}'
output_lr = f'dataset/{types}/LQ'
output_hr = f'dataset/{types}/HQ'
os.makedirs(output_lr, exist_ok = True)
os.makedirs(output_hr, exist_ok = True)
# os.makedirs(output, exist_ok = True)
for path in tqdm(os.listdir(img_dir)[:100], desc = f'quality {quality}', unit = 'img'):
    img_path = os.path.join(img_dir, path)
    # out_path = os.path.join(output, f"{os.path.splitext(path)[0]}.png")
    out_lr_path = os.path.join(output_lr, f"{os.path.splitext(path)[0]}_{quality}.png")
    out_hr_path = os.path.join(output_hr, f"{os.path.splitext(path)[0]}_{quality}.png")
    img = Image.open(img_path)
    h, w = img.size
    # print(h, w)
    out = img.resize((h//4, w//4))
    # out = img
    if quality == 'origin':
        out.save(out_lr_path, format="PNG", optimize=True)
        
    else:
        out.save(out_lr_path, format="JPEG", quality=quality, optimize=True)
    # img.save(out_hr_path, format='PNG', quality=100, optimize=True)
    shutil.copy2(img_path, out_hr_path)

In [ ]:
for types in ['train']:
    # img_dir = f'/kaggle/input/div2k-dataset/DIV2K_{types}_HR/DIV2K_{types}_HR'
    img_dir = f'/kaggle/input/train600/train/images'
    # for quality in [20, 'origin']:
    # for quality in [20]:

    output_lr = f'dataset/{types}/LQ'
    output_hr = f'dataset/{types}/HQ'
    os.makedirs(output_lr, exist_ok = True)
    os.makedirs(output_hr, exist_ok = True)
    for _, path in enumerate(tqdm(os.listdir(img_dir)[:int(len(os.listdir(img_dir)))], desc = f'quality {quality}', unit = 'img')):
        img_path = os.path.join(img_dir, path)
        # out_lr_path = os.path.join(output_lr, f"{os.path.splitext(path)[0]}_{quality}.png")
        # out_hr_path = os.path.join(output_hr, f"{os.path.splitext(path)[0]}_{quality}.png")
        # if i > 1:
            # continue
        if types == 'train' and quality != 'origin':
            patches = 1
        else:
            patches = 1
        for i in range(patches):
            out_lr_path = os.path.join(output_lr, f"{os.path.splitext(path)[0]}_{quality}_{i}.png")
            out_hr_path = os.path.join(output_hr, f"{os.path.splitext(path)[0]}_{quality}_{i}.png")
            img = Image.open(img_path)
            h, w = img.size  
            size = imgsz
            # print(f"Current types: {types}")
            # if types == 'train':
            # # Xác định tọa độ cắt ngẫu nhiên
            #     x = random.randint(0, w - size)
            #     y = random.randint(0, h - size)
                
            #     # Cắt vùng ảnh 196x196
            #     img = img.crop((x, y, x + size, y + size))
                
            # h, w = img.size 
            # print(img.size, out.size)
            out = img.resize((h//4, w//4))

            # out = img
            if quality == 'origin':
                out.save(out_lr_path, format="PNG", optimize=True)
                
            else:
                out.save(out_lr_path, format="JPEG", quality=quality, optimize=True)
            # img.save(out_hr_path, format='PNG', quality=100, optimize=True)
            shutil.copy2(img_path, out_hr_path)
           

In [ ]:
# import shutil
# for types in ['train', 'test']:
#     for qual in ['LQ', 'HQ']:
#         if qual == 'LQ':
#             img_dir = f'/kaggle/input/sr-dataset/_output_/dataset/{quality}/{types}/{qual}'
#         else:
#             img_dir = f'/kaggle/input/sr-dataset/_output_/dataset/{types}/{qual}'
        
#         hr_img_path = f'dataset/{types}/{qual}'
        
#         os.makedirs(hr_img_path, exist_ok = True)
        
#         for path in tqdm(os.listdir(img_dir), unit = 'img'):
#             source_file = os.path.join(img_dir, path)
#             file1 = os.path.join(hr_img_path, path)
#             shutil.copy(source_file, file1)
           

In [ ]:
# import shutil
# shutil.make_archive('dataset', 'zip', 'dataset')

# 2. Tạo Mô hình SR

In [ ]:
import torchvision.transforms.functional as TF

class ImageDataset(Dataset):
    def __init__(self, lr_dir, hr_dir, scale, valid = False):
        self.lr_files = sorted(os.listdir(lr_dir))
        self.hr_files = sorted(os.listdir(hr_dir))
        self.lr_dir = lr_dir
        self.hr_dir = hr_dir
        self.scale = scale
        self.valid = valid

    def __len__(self):
        return len(self.lr_files)

    def __getitem__(self, idx):
        lr_image = Image.open(os.path.join(self.lr_dir, self.lr_files[idx])).convert('RGB')
        hr_image = Image.open(os.path.join(self.hr_dir, self.hr_files[idx])).convert('RGB')
    
        w, h= hr_image.size
        # print(hr_image.size, lr_image.size)
        # if self.valid:
            # lr_image = lr_image.resize((w//self.scale, h//self.scale))
        transform = transforms.Compose([
            transforms.ToTensor()
        ])

        if self.valid:
            def transform_fn(lr_img, hr_img):
                # i, j, h, w = transforms.RandomCrop.get_params(lr_img, output_size=(imgsz, imgsz))
                # lr_img = TF.crop(lr_img, i, j, h, w)
                # hr_img = TF.crop(hr_img, i, j, h, w)
                lr_img = TF.to_tensor(lr_img)
                hr_img = TF.to_tensor(hr_img)
        
                return lr_img, hr_img

        else:
            def transform_fn(lr_img, hr_img):
                # Lấy thông số ngẫu nhiên cho cropping
                i, j, h, w = transforms.RandomCrop.get_params(lr_img, output_size=(imgsz//4, imgsz//4))
                lr_img = TF.crop(lr_img, i, j, h, w)
                hr_img = TF.crop(hr_img, i * scale, j * scale, h * scale, w * scale)
        
                # Áp dụng cùng một phép xoay ngẫu nhiên
                angle = random.uniform(-30, 30)
                lr_img = TF.rotate(lr_img, angle)
                hr_img = TF.rotate(hr_img, angle)
        
                # Áp dụng cùng một phép lật ngang
                if random.random() > 0.5:
                    lr_img = TF.hflip(lr_img)
                    hr_img = TF.hflip(hr_img)
        
                # Áp dụng cùng một phép lật dọc
                if random.random() > 0.5:
                    lr_img = TF.vflip(lr_img)
                    hr_img = TF.vflip(hr_img)
        
                # Chuyển sang tensor
                lr_img = TF.to_tensor(lr_img)
                hr_img = TF.to_tensor(hr_img)
        
                return lr_img, hr_img
        lr_image, hr_image = transform_fn(lr_image, hr_image)
        return lr_image, hr_image
        

# 3. Tạo Hyperparameter

In [ ]:
iqe = IQE().to(device)
isr = HQSR(scale_factor=4, use_canny=True).to(device)

In [ ]:
weight_isr = f'/kaggle/input/tamsadsadas/checkpoint_isr_{imgsz}.pth'
weight_iqe = f'/kaggle/input/tamsadsadas/checkpoint_iqe_{imgsz}.pth'

isr.load_state_dict(torch.load(weight_isr, map_location=device)['model_state_dict'])
iqe.load_state_dict(torch.load(weight_iqe, map_location=device)['model_state_dict'])

In [ ]:
import os
os.environ["TORCH_USE_CUDA_DSA"] = "1"
os.environ["CUDA_LAUNCH_BLOCKING"]="1"

In [ ]:
def calculate_psnr(img1, img2):
    mse = torch.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    max_pixel = 1.0
    psnr = 20 * torch.log10(max_pixel / torch.sqrt(mse))
    return psnr.item()

# 4. Training

In [ ]:
def save_checkpoint(model, optimizer, scheduler, epoch, train_step, test_step, best_loss, path="checkpoint.pth"):
    checkpoint = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'epoch': epoch,
        'train_step': train_step,
        'test_step':test_step,
        'best_loss': best_loss,
    }
    torch.save(checkpoint, path)
    print(f"Checkpoint saved at {path}")

# Hàm tải trạng thái
def load_checkpoint(model, optimizer, scheduler, path="checkpoint.pth"):
    if os.path.isfile(path):
        checkpoint = torch.load(path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        epoch = checkpoint['epoch']
        train_step = checkpoint['train_step']
        val_step = checkpoint['test_step']
        best_loss = checkpoint['best_loss']
        print(f"Checkpoint loaded from {path}")
    else:
        print(f"No checkpoint found at {path}")
        epoch, train_step, val_step, best_loss = 0, 0, 0, float('inf')
    return epoch, train_step, val_step, best_loss

def load_model_state(model, checkpoint_path="checkpoint.pth"):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"Model state loaded from {checkpoint_path}")
    return model

In [ ]:
from torch.amp import autocast, GradScaler
from torchsummary import summary
scaler = GradScaler()

# Khởi tạo dataset và dataloader
# for scale in [2, 3, 4]:
scale = 4
num_iterations= 3e5
train_lr_dir = f'dataset/train/LQ'
train_hr_dir = 'dataset/train/HQ'
# valid_lr_dir = 'dataset/valid/LQ'
# valid_hr_dir = 'dataset/valid/HQ'

valid_lr_dir = 'dataset/test/LQ'
valid_hr_dir = 'dataset/test/HQ'
test_lr_dir = 'dataset/test/LQ'
test_hr_dir = 'dataset/test/HQ'

train_dataset = ImageDataset(train_lr_dir, train_hr_dir, scale=scale)
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)

valid_dataset = ImageDataset(valid_lr_dir, valid_hr_dir, scale=scale, valid=True)
valid_loader = DataLoader(valid_dataset)

test_dataset = ImageDataset(test_lr_dir, test_hr_dir, scale=scale, valid=True)
test_loader = DataLoader(test_dataset)
# print(len(train_loader))
# Khởi tạo mô hình, loss function và optimizer
torch.cuda.empty_cache()

criterion = nn.MSELoss()
optim_iqe = optim.Adam(list(isr.parameters()) + list(iqe.parameters()), lr=1e-5,betas =(0.9, 0.999))
scheduler = optim.lr_scheduler.StepLR(optim_iqe, step_size=10**5, gamma=0.5)

In [ ]:
os.makedirs('outputs/weight', exist_ok=True)
os.makedirs('outputs/path', exist_ok=True)

best_loss= float('inf')

torch.cuda.empty_cache()

train_loss = []
val_loss = []

train_psnr = []
val_psnr = []

log_file = open('iqe.txt', 'a')

start_epoch, train_step, val_step, best_loss = 0, 0, 0, float('inf')
start_epoch, train_step, val_step, best_loss = load_checkpoint(iqe, optim_iqe, scheduler, path=weight_iqe)
# model = load_model_state(
#     model,  checkpoint_path="/kaggle/input/lolv1w/reslle.pth"
# )
patience = 80
epochs_no_improve = 0

num_epochs = math.ceil(num_iterations / len(train_loader))
# num_epochs = 5

In [ ]:
def tensor_batch_to_pil(tensor):    
    if isinstance(tensor, torch.Tensor):
        output_image = tensor.squeeze(0).cpu()  # Loại bỏ batch dimension và chuyển tensor sang CPU
        output_image = transforms.ToPILImage()(output_image)  # Chuyển tensor thành ảnh PIL

    elif isinstance(tensor, Image.Image):
        output_image = tensor
    return output_image

In [ ]:
def concat_triplet_batch(lr, isr_out, iqe_out, out):
    # for i in range(lr.size(0)):
    lr_img = tensor_batch_to_pil(lr)
    isr_img = tensor_batch_to_pil(isr_out)
    iqe_img = tensor_batch_to_pil(iqe_out)
    out_img = tensor_batch_to_pil(out)
    w, h = lr_img.size
    combined = Image.new("RGB", (w*13, h*4))
    combined.paste(lr_img, (0,0))
    combined.paste(isr_img, (w,0))
    combined.paste(iqe_img, (5*w,0))
    combined.paste(out_img, (9*w,0))
    return combined



In [ ]:
torch.cuda.empty_cache()

In [ ]:
for epoch in range(start_epoch, num_epochs):
    iqe.train()
    isr.train()
    epoch_loss, epoch_psnr = 0, 0
    
    start_time = time.time()
    torch.cuda.empty_cache()
    # Training loop for each model
    for i, (lr_images, hr_images) in enumerate(tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch')):
        lr_images = lr_images.to(device)
        hr_images = hr_images.to(device)
        # print(lr_images.shape)

        # # Train model model
        optim_iqe.zero_grad()
        with autocast(device_type='cuda'):
            output_sr = isr(lr_images)
            outputs = iqe(output_sr)
            loss_sr = criterion(output_sr, hr_images)
            loss_iqe = criterion(outputs, hr_images)
            loss = 0.5 * loss_sr + 0.5 * loss_iqe
            # if epoch % 25  == 0 and i == 0:
            #     experiment.log_image(tensor_batch_to_pil(hr_images), name=f"HR_image", step=epoch)
            #     experiment.log_image(tensor_batch_to_pil(output_sr), name=f"ISR output", step=epoch)
            #     experiment.log_image(tensor_batch_to_pil(outputs), name=f"IQE output", step=epoch)


        psnr = calculate_psnr(outputs, hr_images)

        scaler.scale(loss).backward()
        scaler.step(optim_iqe)
        scaler.update()
        scheduler.step()

        epoch_loss += loss.item()
        epoch_psnr += psnr
        
        train_step += 1
        elapsed_time = time.time() - start_training_time
        
        experiment.log_metric("train_loss", loss.item(), step=train_step)
        experiment.log_metric("train_psnr", psnr, step=train_step)

    # Average losses and PSNRs
    avg_epoch_loss = epoch_loss / len(train_loader)
    avg_epoch_psnr = epoch_psnr / len(train_loader)


    # Validation for all models
    iqe.eval()
    isr.eval()
    epoch_val_psnr = 0
    epoch_val_loss = 0
    with torch.no_grad():
        for i, (lr_images, hr_images) in enumerate(tqdm(valid_loader, desc=f'Val Epoch {epoch+1}: ', unit='batch')):
            lr_images = lr_images.cuda()
            hr_images = hr_images.cuda()

            # # Validate model
            output_sr = isr(lr_images)
            outputs = iqe(output_sr)
            loss_sr = criterion(output_sr, hr_images)
            loss_iqe = criterion(outputs, hr_images)
            loss = 0.5 * loss_sr + 0.5 * loss_iqe
            
            psnr = calculate_psnr(outputs, hr_images)
            epoch_val_loss += loss.item()
            epoch_val_psnr += psnr
            experiment.log_metric("val_loss", loss.item(), step=val_step)
            experiment.log_metric("val_psnr", psnr, step=val_step)
            if epoch % 1  == 0 and i == 0:
                # experiment.log_image(tensor_batch_to_pil(lr_images), name=f"LR_image", step=epoch)
                # experiment.log_image(tensor_batch_to_pil(hr_images), name=f"HR_image", step=epoch)
                # experiment.log_image(tensor_batch_to_pil(output_sr), name=f"ISR output", step=epoch)
                # experiment.log_image(tensor_batch_to_pil(outputs), name=f"IQE output", step=epoch)
                experiment.log_image(concat_triplet_batch(lr_images, output_sr, outputs, hr_images), name="Comparison", step=epoch)
            val_step += 1
            
    val_psnr_epoch = epoch_val_psnr / len(valid_loader)
    val_loss_epoch = epoch_val_loss / len(valid_loader)
    experiment.log_metrics({'avg_train_loss':avg_epoch_loss, 'avg_val_loss':val_loss_epoch}, step=epoch)
    experiment.log_metrics({'avg_train_psnr':avg_epoch_psnr, 'avg_val_psnr':val_psnr_epoch}, step=epoch)

    # Save best model
    
    if epoch % 10 == 0:
        save_checkpoint(iqe, optim_iqe, scheduler, epoch, train_step, val_step, best_loss, path=f"outputs/path/iqe_{epoch}.pth")
        save_checkpoint(isr, optim_iqe, scheduler, epoch, train_step, val_step, best_loss, path=f"outputs/path/isr_{epoch}.pth")
    save_checkpoint(iqe, optim_iqe, scheduler, epoch, train_step, val_step, best_loss, path=f"outputs/checkpoint_iqe.pth")
    save_checkpoint(isr, optim_iqe, scheduler, epoch, train_step, val_step, best_loss, path=f"outputs/checkpoint_isr.pth")
        
    print(f"Epoch [{epoch+1}/{num_epochs}] completed: training loss: {avg_epoch_loss:.6f}, validation loss: {val_loss_epoch:.6f}, Training PSNR: {avg_epoch_psnr:.2f}, Validation PSNR: {val_psnr_epoch:.2f},")
    log_file.write(f"Epoch {epoch+1}:  training loss: {avg_epoch_loss:.6f}, validation loss: {val_loss_epoch:.6f}, Training PSNR: {avg_epoch_psnr:.2f}, Validation PSNR: {val_psnr_epoch:.2f}\n")
    if val_loss_epoch < best_loss:
        best_loss = val_loss_epoch
        save_checkpoint(iqe, optim_iqe, scheduler, epoch, train_step, val_step, best_loss, path=f"outputs/weight/best_iqe_{quality}.pth")
        save_checkpoint(isr, optim_iqe, scheduler, epoch, train_step, val_step, best_loss, path=f"outputs/weight/best_isr_{quality}.pth")

        print(f"Saved model with Val loss {best_loss:.4f}")
        log_file.write(f"Saved model with Val loss {best_loss:.4f}\n")
    log_file.flush()
    if elapsed_time >= max_training_time:
            print("Training stopped due to time limit.")
            save_checkpoint(iqe, optim_iqe, scheduler, epoch, train_step, val_step, best_loss, path="outputs/checkpoint_iqe.pth")
            save_checkpoint(isr, optim_iqe, scheduler, epoch, train_step, val_step, best_loss, path="outputs/checkpoint_isr.pth")
            break
experiment.end()


# 5. Testing

In [ ]:
def load_weight(model, path="checkpoint.pth"):
    if os.path.isfile(path):
        checkpoint = torch.load(path)
        model.load_state_dict(checkpoint['model_state_dict'])
        
    else:
        print(f"No checkpoint found at {path}")
        return 0, 0, float('-inf')

In [ ]:
# load_check(f'outputs/best_esr_sobel_{quality}.pth', map_location=device))
load_weight(iqe, path=f'outputs/best_iqe_{quality}.pth')
